# Sentiment Analysis dari Instagram comments calon Presiden 2023

<ul>
    <li>
    Aurelius Ivan Wijaya (00000054769)
    </li>
    <li>
    Rajendra Abhinaya (00000060445)
    </li>
    <li>
    Maecyntha Irelynn Tantra (00000055038)
    </li>
    <li>
    Patricia theodora (00000054093)
    </li>
<ul>

# Data Collection

# Data Integration

## EDA

# Pre-Proccessing

## Case Folding

## Punctuation & Number & Whitespace Removal

## Stopwords Removal

## Stemming / Lemmatization

## Text Normalization / Noise Removal

## Tokenization

## SMOTE

## K-Fold Cross Validation

# Modeling

# Model Evaluation